In [2]:
import geopandas as gpd

## Open GIS Data

In [63]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [64]:
states_df = states_df[
    ~states_df["STUSPS"].isin(["GU", "PR", "HI", "AK", "DC", "VI", "MP"])
]

In [66]:
states_df = states_df.sort_values("AWATER")
states_df = states_df.reset_index()

## Apply theorem 

In [67]:
colors = ["#FF0000", "#0000FF", "#00FF00", "#FFFF00"]

In [68]:
neighbors = {}
for idx in range(len(states_df)):
    geometry = states_df.iloc[idx]["geometry"]
    neighbors[idx] = states_df[states_df.geometry.touches(geometry)].index.tolist()

In [69]:
states_df["color"] = None

In [71]:
for idx in states_df.index:
    neighbor_colors = {
        states_df.loc[neighbor, "color"]
        for neighbor in neighbors[idx]
        if states_df.loc[neighbor, "color"] is not None
    }
    available_colors = [color for color in colors if color not in neighbor_colors]
    states_df.at[idx, "color"] = available_colors[0] if available_colors else None

## Open Dollar General Data

In [72]:
file_path = "data/stores.gpkg"
stores_df = gpd.read_file(file_path)

In [73]:
states_color_df = states_df[["STUSPS", "color"]].rename(columns={"STUSPS": "STATE"})

In [75]:
stores_with_colors_gdf = stores_df.merge(states_color_df, on="STATE", how="left")

In [76]:
stores_with_colors_gdf.to_file("data/states_colors.gpkg")